In [ ]:
import { display } from "tslab";
import { readFileSync } from "fs";

const css = readFileSync("../style.css", "utf8");
display.html(`<style>${css}</style>`);

In [ ]:
import { RecursiveSet, Value } from "recursive-set";
import { DFA, NFA, State, Char, key, DFAState } from "./01-NFA-2-DFA";

### Utility: Deterministic Set Formatting

The helper function `pySetStr` generates a string representation of a `RecursiveSet` (or standard `Set`) that mimics Python's set formatting `{a, b, c}`.

**Why is this necessary?**
1.  **Readability:** It provides a compact view of sets, which is crucial when states themselves are sets of numbers.
2.  **Determinism:** It sorts the elements before stringification. This ensures that `{1, 2}` and `{2, 1}` always produce the exact same output string, which is vital for reproducible tests and debugging.

In [ ]:
function pySetStr(s: RecursiveSet<Value> | Set<Value>): string {
    let elements: Value[];

    if (s instanceof RecursiveSet) {
        if (s.isEmpty()) return "{}";
        elements = s.raw as Value[];
    } else {
        if (s.size === 0) return "{}";
        elements = Array.from(s);
    }

    const sortedStrings = elements.map(String).sort();
    return `{${sortedStrings.join(", ")}}`;
}

### DFA Text Representation

The function `dfa2string` converts a deterministic finite automaton (DFA) into a human-readable text format.

**Key Logic:**
* **State Renaming:** Since DFA states can be complex objects (like `RecursiveSet<number>`), we map them to simple names $S_0, S_1, \dots, S_n$ for clarity.
* **State Encoding:** We output a legend mapping each $S_i$ back to its original set value (using `pySetStr`).
* **Transitions:** We list all transitions $\delta(S_i, c) = S_j$.

**Implementation Note:**
We use `state.toString()` as the key for our mapping to ensure robust lookups, avoiding reference identity issues common with object-based sets.

In [ ]:
function dfa2string(dfa: DFA): string {
    const { Q, Sigma, delta, q0, A } = dfa;
    let result: string = "";

    const stateToName = new Map<string, string>();
    const statesList = Q.raw;

    let n = 0;
    for (const q of statesList) {
        stateToName.set(q.toString(), `S${n++}`);
    }

    result += `states: {S0, ..., S${n - 1}}\n\n`;

    const startName = stateToName.get(q0.toString()) ?? "UNKNOWN";
    result += `start state: ${startName}\n\n`;

    result += "state encoding:\n";
    for (const q of statesList) {
        result += `${stateToName.get(q.toString())} = ${pySetStr(q)}\n`;
    }

    result += "\ntransitions:\n";
    for (const q of statesList) {
        const sourceName = stateToName.get(q.toString())!;

        for (const rawC of Sigma) {
            const c = rawC as string;
            const k = key(q, c);

            const target = delta.get(k);

            if (target) {
                const targetName = stateToName.get(target.toString());

                if (targetName) {
                    result += `delta(${sourceName}, ${c}) = ${targetName}\n`;
                }
            }
        }
    }

    result += "\nset of accepting states: {";
    const acceptingNames: string[] = [];

    for (const q of statesList) {
        if (A.has(q)) {
            acceptingNames.push(stateToName.get(q.toString())!);
        }
    }
    result += acceptingNames.join(", ");
    result += "}\n";

    return result;
}

### DFA Visualization (DOT Format)

The function `dfa2dot` generates a Graphviz DOT definition for the DFA.

**Features:**
* **Layout:** Uses `rankdir=LR` for a Left-to-Right flow.
* **Start State:** Marked by an invisible "ghost node" pointing to the actual start state.
* **Accepting States:** Drawn with a double periphery (`peripheries="2"`).
* **Return Value:** Returns both the DOT string and the `statesToNames` map, allowing subsequent code to reference the generated state names.

In [ ]:
function dfa2dot(dfa: DFA): {
    dot: string;
    statesToNames: Map<DFAState, string>;
} {
    const { Q, Sigma, delta, q0, A } = dfa;
    const lines: string[] = [];

    lines.push('digraph "Deterministic FSM" {');
    lines.push("  rankdir=LR;");

    const stateToNameStr = new Map<string, string>();
    const stateToNameObj = new Map<DFAState, string>();
    const statesList = Q.raw;

    let n = 0;
    for (const q of statesList) {
        const name = `S${n++}`;
        stateToNameStr.set(q.toString(), name);
        stateToNameObj.set(q, name);
    }

    const startName = stateToNameStr.get(q0.toString()) ?? "ERR";

    lines.push(
        '  "start_ghost" [label="", width="0.1", height="0.1", style="filled", color="blue"];',
    );
    lines.push(`  "start_ghost" -> "${startName}";`);

    for (const q of statesList) {
        const name = stateToNameStr.get(q.toString())!;
        if (A.has(q)) {
            lines.push(`  "${name}" [peripheries="2"];`);
        } else {
            lines.push(`  "${name}";`);
        }
    }

    for (const q of statesList) {
        const sourceName = stateToNameStr.get(q.toString())!;

        for (const rawC of Sigma) {
            const c = rawC as string;
            const target = delta.get(key(q, c));

            if (target) {
                const targetName = stateToNameStr.get(target.toString());
                if (targetName) {
                    lines.push(
                        `  "${sourceName}" -> "${targetName}" [label="${c}"];`,
                    );
                }
            }
        }
    }

    lines.push("}");
    return { dot: lines.join("\n"), statesToNames: stateToNameObj };
}

### NFA Text Representation

The function `nfa2string` converts a non-deterministic finite automaton (NFA) to text.

**Differences from DFA:**
* **Multiple Targets:** A transition `[q, c]` can map to a *set* of target states. We iterate through all targets.
* **Epsilon Transitions:** We specifically check for transitions labeled with $\varepsilon$ (epsilon) or empty string and list them as `[q, ""] |-> p`.

In [ ]:
function nfa2string(nfa: NFA): string {
    const { Q, Sigma, delta, q0, A } = nfa;
    let result: string = "";

    result += `states: ${pySetStr(Q)}\n\n`;
    result += `start state: ${q0}\n\n`;
    result += "transitions:\n";

    const sortedStates = Array.from(Q).map(String).sort();
    const sortedSigma = Array.from(Sigma).map(String).sort();

    for (const qStr of sortedStates) {
        const q = (isNaN(Number(qStr)) ? qStr : Number(qStr)) as State;

        for (const c of sortedSigma) {
            const targets = delta.get(key(q, c));
            if (targets && !targets.isEmpty()) {
                for (const p of targets) {
                    result += `[${q}, ${c}] |-> ${p}\n`;
                }
            }
        }

        const targetsEps = delta.get(key(q, "ε"));
        if (targetsEps && !targetsEps.isEmpty()) {
            for (const p of targetsEps) {
                result += `[${q}, ""] |-> ${p}\n`;
            }
        }
    }

    result += `\nset of accepting states: ${pySetStr(A)}\n`;
    return result;
}

### NFA Visualization (DOT Format)

The function `nfa2dot` visualizes an NFA.

**Visual Cues:**
* **Epsilon Edges:** Transitions labeled $\varepsilon$ are drawn with a lower edge weight (`weight="0.1"`) to encourage Graphviz to place them distinctly, often hinting at the "instant" nature of the transition.
* **Direct Mapping:** Since NFA states are typically simple primitives (numbers/strings), we use their string representation directly as node names (unlike the DFA remapping).

In [ ]:
function nfa2dot(nfa: NFA): string {
    const { Q, Sigma, delta, q0, A } = nfa;
    const lines: string[] = [];

    lines.push('digraph "Non-Deterministic FSM" {');
    lines.push("  rankdir=LR;");

    const startName = q0.toString();
    const statesList = Array.from(Q);

    lines.push(
        '  "start_ghost" [label="", width="0.1", height="0.1", style="filled", color="blue"];',
    );
    lines.push(`  "start_ghost" -> "${startName}";`);

    for (const q of statesList) {
        if (A.has(q)) {
            lines.push(`  "${q}" [peripheries="2"];`);
        } else {
            lines.push(`  "${q}";`);
        }
    }

    for (const q of statesList) {
        const targets = delta.get(key(q, "ε"));
        if (targets) {
            for (const p of targets) {
                lines.push(`  "${q}" -> "${p}" [label="ε", weight="0.1"];`);
            }
        }
    }

    for (const q of statesList) {
        for (const rawC of Sigma) {
            const c = rawC as string;
            const targets = delta.get(key(q, c));
            if (targets) {
                for (const p of targets) {
                    lines.push(
                        `  "${q}" -> "${p}" [label="${c}", weight="10"];`,
                    );
                }
            }
        }
    }

    lines.push("}");
    return lines.join("\n");
}